# Data Cleaning and Preprocessing

**Project:** Trust, Ethical Concerns, and Usage Patterns of ChatGPT  
**Team:** Team 5  
**Date:** November 2025

This notebook loads the raw dataset, performs cleaning, creates composite scores, and saves cleaned data.

## 1. Setup and Imports

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set style for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## 2. Load Raw Data

In [ ]:
# Load dataset
df = pd.read_excel('../data/raw.xlsx')

print(f"Dataset shape: {df.shape}")
print(f"Rows: {len(df):,}")
print(f"Columns: {len(df.columns)}")

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Display column names
print("All columns:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:3d}. {col}")

## 3. Data Quality Checks

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

if duplicates > 0:
    df = df.drop_duplicates()
    print(f"✅ Removed {duplicates} duplicates")
    print(f"New shape: {df.shape}")

In [ ]:
# Check missing data
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing': missing,
    'Percent': missing_pct
}).sort_values('Percent', ascending=False)

print("Top 20 columns with missing data:")
print(missing_df.head(20))

## 4. Create Composite Scores

### 4.1 Perceived Capabilities (Q22a-j)

In [ ]:
# Define capability items
capability_cols = [f'Q22{x}' for x in ['a','b','c','d','e','f','g','h','i','j']]
existing_caps = [col for col in capability_cols if col in df.columns]

print(f"Capability columns found: {len(existing_caps)}/10")
print(existing_caps)

# Create composite score
if existing_caps:
    df['capabilities_score'] = df[existing_caps].mean(axis=1)
    
    print(f"\n✅ Capabilities score created")
    print(f"Mean: {df['capabilities_score'].mean():.2f}")
    print(f"SD: {df['capabilities_score'].std():.2f}")
    print(f"Range: [{df['capabilities_score'].min():.2f}, {df['capabilities_score'].max():.2f}]")

In [ ]:
# Visualize distribution
plt.figure(figsize=(10, 5))
plt.hist(df['capabilities_score'].dropna(), bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Capabilities Score (1-5)')
plt.ylabel('Frequency')
plt.title('Distribution of Perceived Capabilities Score')
plt.axvline(df['capabilities_score'].mean(), color='red', linestyle='--', label='Mean')
plt.legend()
plt.show()

### 4.2 Ethical Concerns (Q28a-i)

In [ ]:
# Define ethics items
ethics_cols = [f'Q28{x}' for x in ['a','b','c','d','e','f','g','h','i']]
existing_ethics = [col for col in ethics_cols if col in df.columns]

print(f"Ethics columns found: {len(existing_ethics)}/9")
print(existing_ethics)

# Create composite score
if existing_ethics:
    df['ethics_concerns_score'] = df[existing_ethics].mean(axis=1)
    
    print(f"\n✅ Ethical concerns score created")
    print(f"Mean: {df['ethics_concerns_score'].mean():.2f}")
    print(f"SD: {df['ethics_concerns_score'].std():.2f}")
    print(f"Range: [{df['ethics_concerns_score'].min():.2f}, {df['ethics_concerns_score'].max():.2f}]")

In [ ]:
# Visualize distribution
plt.figure(figsize=(10, 5))
plt.hist(df['ethics_concerns_score'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='orange')
plt.xlabel('Ethical Concerns Score (1-5)')
plt.ylabel('Frequency')
plt.title('Distribution of Ethical Concerns Score')
plt.axvline(df['ethics_concerns_score'].mean(), color='red', linestyle='--', label='Mean')
plt.legend()
plt.show()

### 4.3 Attitudes (Q21a-d)

In [ ]:
# Define attitude items
attitude_cols = [f'Q21{x}' for x in ['a','b','c','d']]
existing_attitudes = [col for col in attitude_cols if col in df.columns]

print(f"Attitude columns found: {len(existing_attitudes)}/4")
print(existing_attitudes)

# Create composite score
if existing_attitudes:
    df['attitudes_score'] = df[existing_attitudes].mean(axis=1)
    
    print(f"\n✅ Attitudes score created")
    print(f"Mean: {df['attitudes_score'].mean():.2f}")
    print(f"SD: {df['attitudes_score'].std():.2f}")
    print(f"Range: [{df['attitudes_score'].min():.2f}, {df['attitudes_score'].max():.2f}]")

## 5. Create Usage Labels

In [ ]:
# Map usage frequency
if 'Q8' in df.columns:
    usage_map = {1.0: 'Daily', 2.0: 'Weekly', 3.0: 'Monthly'}
    df['usage_label'] = df['Q8'].map(usage_map)
    
    print("Usage Frequency Distribution:")
    print(df['usage_label'].value_counts())
    
    # Visualize
    plt.figure(figsize=(8, 5))
    df['usage_label'].value_counts().plot(kind='bar', color=['green', 'blue', 'orange'])
    plt.xlabel('Usage Frequency')
    plt.ylabel('Number of Students')
    plt.title('ChatGPT Usage Frequency Distribution')
    plt.xticks(rotation=0)
    plt.show()

## 6. Verify Key Variables

In [ ]:
# Check key variables
key_vars = {
    'Trust (Q15)': 'Q15',
    'Recommendation (Q16)': 'Q16',
    'Usage (Q8)': 'Q8',
    'Capabilities': 'capabilities_score',
    'Ethics Concerns': 'ethics_concerns_score',
    'Attitudes': 'attitudes_score'
}

print("Key Variables Summary:")
print("="*60)

for name, var in key_vars.items():
    if var in df.columns:
        missing = df[var].isna().sum()
        missing_pct = (missing / len(df)) * 100
        print(f"{name:20s} - Missing: {missing:5d} ({missing_pct:5.1f}%)")
    else:
        print(f"{name:20s} - ❌ NOT FOUND")

## 7. Reliability Analysis (Cronbach's Alpha)

In [ ]:
try:
    import pingouin as pg
    
    print("Cronbach's Alpha (Reliability):")
    print("="*60)
    
    # Capabilities
    if existing_caps:
        alpha_cap = pg.cronbach_alpha(data=df[existing_caps].dropna())
        print(f"Capabilities Scale: α = {alpha_cap[0]:.3f} {'✅' if alpha_cap[0] > 0.70 else '⚠️'}")
    
    # Ethics
    if existing_ethics:
        alpha_eth = pg.cronbach_alpha(data=df[existing_ethics].dropna())
        print(f"Ethics Scale:       α = {alpha_eth[0]:.3f} {'✅' if alpha_eth[0] > 0.70 else '⚠️'}")
    
    # Attitudes
    if existing_attitudes:
        alpha_att = pg.cronbach_alpha(data=df[existing_attitudes].dropna())
        print(f"Attitudes Scale:    α = {alpha_att[0]:.3f} {'✅' if alpha_att[0] > 0.70 else '⚠️'}")
    
    print("\nNote: α > 0.70 indicates acceptable reliability")
    
except ImportError:
    print("⚠️ pingouin not installed. Run: pip install pingouin")
    print("   Skipping reliability analysis for now.")

## 8. Save Cleaned Data

In [ ]:
# Save cleaned dataset
output_path = '../data/cleaned/cleaned_data.csv'

df.to_csv(output_path, index=False)

print(f"✅ Cleaned data saved to: {output_path}")
print(f"   Shape: {df.shape}")
print(f"   Size: {Path(output_path).stat().st_size / 1024 / 1024:.1f} MB")

## 9. Summary

In [ ]:
print("="*80)
print("DATA CLEANING SUMMARY")
print("="*80)
print(f"""
Final Dataset:
- Rows: {len(df):,}
- Columns: {len(df.columns)}

Composite Scores Created:
- Capabilities (Q22a-j): {'✅' if 'capabilities_score' in df.columns else '❌'}
- Ethical Concerns (Q28a-i): {'✅' if 'ethics_concerns_score' in df.columns else '❌'}
- Attitudes (Q21a-d): {'✅' if 'attitudes_score' in df.columns else '❌'}

Key Variables:
- Trust (Q15): {'✅' if 'Q15' in df.columns else '❌'}
- Usage (Q8): {'✅' if 'Q8' in df.columns else '❌'}
- Recommendation (Q16): {'✅' if 'Q16' in df.columns else '❌'}

✅ Data cleaning complete!
""")

## Next Steps

1. Run **02_descriptive_statistics.ipynb** for summary stats
2. Run **03_correlation_analysis.ipynb** for RQ3
3. Run **04_regression_analysis.ipynb** for RQ1